### Here I will collect weather informations about the 35 destinations:
- for the next 7 days

In [1]:
import pandas as pd
import requests
import time
import json
import os  

In [2]:
# preparing dataframe creation

list_columns = ['cityname', 'temperature_j1', 'precipitation_j1', 'weathercode_j1',
       'temperature_j2', 'precipitation_j2', 'weathercode_j2',
       'temperature_j3', 'precipitation_j3', 'weathercode_j3',
       'temperature_j4', 'precipitation_j4', 'weathercode_j4',
       'temperature_j5', 'precipitation_j5', 'weathercode_j5',
       'temperature_j6', 'precipitation_j6', 'weathercode_j6',
       'temperature_j7', 'precipitation_j7', 'weathercode_j7']



In [3]:
# The API (open source) I will request to obtain weather informations

def get_weather_from_coordinates(lat,lon):
    latitude = lat
    longitude = lon
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m&hourly=precipitation&hourly=weathercode"
    response_api = requests.get(url)
    print(f'{url}')
    return response_api

In [4]:
# For my final dataframe, I select temperature, precipitation and weather code for 
# each day at 12h00

def create_mydict(json_returned_by_api):
    list_index = [12,36,60,84,108,132,166]
    my_dict = {}

    for count, value in enumerate(list_index, start=1):
        my_dict[f'temperature_j{count}'] = [result_json.get('hourly').get('temperature_2m')[value]]
        my_dict[f'precipitation_j{count}'] = [result_json.get('hourly').get('precipitation')[value]]
        my_dict[f'weathercode_j{count}'] = [result_json.get('hourly').get('weathercode')[value]]
    return my_dict

### Open and read the CSV File :
- dict_gps.csv
- created during get_gps_coordinates_35_destinations.ipynb

In [5]:
path = os.getcwd()
path = str(os.path.abspath(os.path.join(path, os.pardir))) 
path = path + r'\get_gps_coordinates_35_destinations\dict_gps'
with open(path,'r') as f:
    data = json.load(f)

list_city = data
list_keys = list_city.keys()

### Creating the dataframe :
- with 7 days weather 
- for each 35 destinations

In [6]:
count = 1
list_cityname = []
list_coordinates = []
df = pd.DataFrame(columns=list_columns)


for value in list_keys:
    
    print(count)
    time.sleep(2)
    latitude = list_city.get(value).get('latitude')
    longitude = list_city.get(value).get('longitude')

    list_cityname.append(value)
    list_coordinates.append((latitude,longitude))

    result = get_weather_from_coordinates(latitude, longitude)
    result_json = result.json()

    my_dict = create_mydict(result_json)
    df_loop = pd.DataFrame(my_dict)
    df_final = pd.concat([df, df_loop])
    df = df_final

    count +=1

df_final['cityname'] = list_cityname
df_final['coordinates'] = list_coordinates
df_final.reset_index(inplace=True)
df_final = df_final.drop(labels='index', axis = 1)


1
https://api.open-meteo.com/v1/forecast?latitude=48.6359541&longitude=-1.511459954959514&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
2
https://api.open-meteo.com/v1/forecast?latitude=48.649518&longitude=-2.0260409&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
3
https://api.open-meteo.com/v1/forecast?latitude=49.2764624&longitude=-0.7024738&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
4
https://api.open-meteo.com/v1/forecast?latitude=49.4938975&longitude=0.1079732&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
5
https://api.open-meteo.com/v1/forecast?latitude=49.4404591&longitude=1.0939658&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
6
https://api.open-meteo.com/v1/forecast?latitude=48.8588897&longitude=2.3200410217200766&hourly=temperature_2m&hourly=precipitation&hourly=weathercode
7
https://api.open-meteo.com/v1/forecast?latitude=49.8941708&longitude=2.2956951&hourly=temperature_2m&hourly=precipitation

In [7]:
df_final.head(5)

,cityname,temperature_j1,precipitation_j1,weathercode_j1,temperature_j2,precipitation_j2,weathercode_j2,temperature_j3,precipitation_j3,weathercode_j3,...,temperature_j5,precipitation_j5,weathercode_j5,temperature_j6,precipitation_j6,weathercode_j6,temperature_j7,precipitation_j7,weathercode_j7,coordinates
0,Mont+Saint+Michel,15.6,0.0,3,11.9,0.6,61,9.7,0.0,2,...,12.9,0.0,3,13.7,0.0,3,11.6,0.0,3,"(48.6359541, -1.511459954959514)"
1,St+Malo,15.9,0.0,3,11.3,0.7,53,10.0,0.0,0,...,13.1,0.0,3,13.5,0.0,3,11.5,0.1,61,"(48.649518, -2.0260409)"
2,Bayeux,16.2,0.0,3,12.0,0.3,61,9.6,0.0,1,...,13.1,0.0,3,12.7,0.0,3,11.4,0.1,61,"(49.2764624, -0.7024738)"
3,Le+Havre,15.5,0.0,2,11.7,0.5,61,9.3,0.1,3,...,12.7,0.0,3,11.2,0.0,3,11.9,0.1,3,"(49.4938975, 0.1079732)"
4,Rouen,16.2,0.0,3,11.4,0.5,61,9.8,0.0,2,...,11.3,0.0,3,10.5,0.0,3,11.8,0.1,3,"(49.4404591, 1.0939658)"


In [8]:
df_final.shape

(35, 23)

In [9]:
df_final.to_csv('weather_to_sort.csv')